In [ ]:
import os

from veil.veil_client import VeilClient
from veil.constants import NetworkId, NETWORK_INFO
from utils.logutils import setup_logger

LOGGER = setup_logger("VeilPlayground")

NETWORK_ID = NetworkId.KOVAN
PRIVATE_KEY = os.environ.get("PRIVATE_KEY")
INFURA_API_KEY = os.environ.get("INFURA_API_KEY")
WEB3_RPC_URL = "https://{}.infura.io/{}".format(NETWORK_ID.name.lower(), INFURA_API_KEY)

client = VeilClient(
    network_id=NETWORK_ID.value,
    web3_rpc_url=WEB3_RPC_URL,
    private_key=PRIVATE_KEY,
)

MY_ADDRESS = client.account_address

print(
    "NETWORK:",  NETWORK_ID,
    "\nRPC_URL", WEB3_RPC_URL,
    "\nADDRESS IN USE:", MY_ADDRESS,
    "\nETH BALANCE:", client.get_eth_balance()
)

print(client)
print("Signing into veil...")
print(client.authenticate())

In [ ]:
client.get_eth_balance()

In [ ]:
markets = client.get_markets(
#     channel="btc", 
    status="open",
#     page=0
)
print(len(markets))
markets[0]

In [ ]:
print(set([x["type"] for x in markets]))
print(set([x["channel"] for x in markets]))

In [ ]:
market = client.get_market(markets[0]["slug"])
market

In [ ]:
from veil.veil_client import TokenType, OrderSide

bids_long = client.get_bids(
    market_slug=market["slug"],
    token_type=TokenType.LONG,
)
bids_short = client.get_bids(
    market_slug=market["slug"],
    token_type=TokenType.SHORT,
)
asks_long = client.get_asks(
    market_slug=market["slug"],
    token_type=TokenType.LONG,
)
asks_short = client.get_asks(
    market_slug=market["slug"],
    token_type=TokenType.SHORT,
)
bids_long, bids_short, asks_long, asks_short

In [ ]:
client.get_order_fills(
    market_slug=market["slug"],
    token_type=TokenType.SHORT,
)